# Week 3: Multiview Reconstruction

# Week 2: Stereo Estimation

If you are working in Colab,
*   Open Files from left
*   Drag and drop `Week3_MVS.zip` there (or click upload icon on top left). Upload can take a few minutes.
*   Unzip the file by running the following cell.


In [ ]:
!unzip Week3_MVS.zip
%cd Week3_MVS

*   If you get disconnected from the runtime, you might need to upload this file again. Also, note that this is valid for other local files too eg. repos that have been cloned, files generated during execution etc. In short, if you see that your files are gone after a while, just run the cell that generates, clones, etc. the missing files again.

## Part A: More on Projections

### Question 1: Transformations

Derive the matrices $M \in SE(3) \subset \mathbb{R}^{4 \times 4}$ representing the following transformations:

* Translation by the vector $T \in \mathbb{R}^3$
* Rotation by the rotation matrix $R \in \mathbb{R}^{3 \times 3}$
* Rotation by $R$ followed by the translation $T$
* Translation by $T$ followed by the rotation $R$

**Hint:** Remember that we can write the transformation matrix $M$ for a given rotation matrix
$R = \begin{bmatrix}
r_{11} & r_{12} & r_{13} \\
r_{21} & r_{22} & r_{23} \\
r_{31} & r_{32} & r_{33}
\end{bmatrix}$ and a translation vector $T = \begin{bmatrix}
t_x \\
t_y \\
t_z
\end{bmatrix}$ as follows:
$M = 
\begin{pmatrix}
R & T \\
0 & 1
\end{pmatrix}
=
\begin{bmatrix}
r_{11} & r_{12} & r_{13} & t_x \\
r_{21} & r_{22} & r_{23} & t_y \\
r_{31} & r_{32} & r_{33} & t_z \\
0 & 0 & 0 & 1 \\
\end{bmatrix}$

**Answer**: 

### Question 2: Scale Ambiguity

A classic ambiguity of the perspective projection is that one cannot tell an object from another object that is exactly twice as big but twice as far. Explain why this is true.

**Hint:** Let $P = (X, Y, Z)$ be a point on the smaller object and $P' = (X', Y', Z')$ a point on the larger object. Define $X' = 2X, Y' = 2Y, Z' = 2Z$ and perpective projection as a function $p = \pi(P)$. How does $\pi$ transform the world coordinate $P$ to image coordinate $p$ according to perspective projection? Repeat the same for $P'$ and $p'$.

**Answer**:

## Part B: Rotating Objects

Write a function that rotates the model around its center (i.e. the mean of its vertices) for given rotation angles $\alpha,~\beta,~\gamma$ around the x-, y- and z-axis. Use homogeneous coordinates and describe the overall transformation by a single matrix. 

The rotation matrices around the respective axes are as follows:

$R_x = \begin{bmatrix}
1 & 0 & 0 \\
0 & \text{cos}~\alpha & -\text{sin}~\alpha \\
0 & \text{sin}~\alpha & \text{cos}~\alpha
\end{bmatrix}
~~~%
R_y = \begin{bmatrix}
\text{cos}~\beta & 0 & \text{sin}~\beta \\
0 & 1 & 0 \\
-\text{sin}~\beta & 0 & \text{cos}~\beta
\end{bmatrix}
~~~%
R_z = \begin{bmatrix}
\text{cos}~\gamma & -\text{sin}~\gamma & 0 \\
\text{sin}~\gamma & \text{cos}~\gamma & 0 \\
0 & 0 & 1
\end{bmatrix}
$

Rotate the model first 50 degrees around the x-axis and then 25 degrees around the z-axis. Now start again by doing the same rotation around the z-axis first followed by the x-axis rotation. What do you observe?

In [ ]:
import open3d as o3d
import numpy as np

def deg2rad(angleInDegrees):
    angleInRadians = (np.pi/180) * angleInDegrees
    return angleInRadians

def rotation(V, alpha_degree, beta_degree, gamma_degree, order):

    # Compute mean of vertices for vertex list V 
    center = None
    
    #Create a translation matrix T as a 4x4 matrix that translates the model to the point (0,0,0) 
    T = None
    
    #Create a translation matrix T_back as a 4x4 matrix that translates back to the center of V
    T_back = None
    
    # convert degrees to radians
    alpha, beta, gamma = None, None, None
    
    # Rotation matrices in homegeneuous coordinate
    R_x = None
    R_y = None
    R_z = None
    
    # Calculate Overall transformation matrix G (using T, T_back, R_x, R_y, R_z, order)
    # order determines the order of applying the rotations. 
    # e.g. if order=='xyz' first rotate around the x axis, then around y axis and then around the z axis
    G = None
            
    # Homogeneous coordinates of V
    Vh = None
    
    # Apply the transformation to the vertices (using G and Vh)
    Wh = None
    # Go back from homogenous  to 3D coordinates
    W = None
    return W
         
# load the model
mesh = o3d.io.read_triangle_mesh("data/model.off")
V = np.asarray(mesh.vertices)

# display the model
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh])

# rotate the model (first around the x axis)
W = rotation(V, 50, 0, 25, order='xyz')
rotated_mesh = o3d.geometry.TriangleMesh(vertices=o3d.utility.Vector3dVector(W), triangles=mesh.triangles)

# display the rotated model
rotated_mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([rotated_mesh])

# rotate the model (first around the z axis)
W = rotation(V, 50, 0, 25, order='zyx')
rotated_mesh = o3d.geometry.TriangleMesh(vertices=o3d.utility.Vector3dVector(W), triangles=mesh.triangles)

# display the rotated model
rotated_mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([rotated_mesh])

print('done!')

## Part C: pykitti

There is a nice repository which serves as a development kit for KITTI in python: [pykitti](https://github.com/utiasSTARS/pykitti)

Install it and repeat the steps below with the provided sequence to see what kind of properties of the dataset is available with pykitti. After that, you will compute stereo as you did last week, this time by using pykitti.

In [ ]:
import pykitti
import numpy as np

basedir = 'data/KITTI-Raw'
date = '2011_09_26'
drive = '0079'

# The 'frames' argument is optional - default: None, which loads the whole dataset.
# Calibration, timestamps, and IMU data are read automatically. 
# Camera and velodyne data are available via properties that create generators
# when accessed, or through getter methods that provide random access.
dataset = pykitti.raw(basedir, date, drive, frames=range(0, 50, 5))

# dataset.calib:         Calibration data are accessible as a named tuple
# dataset.timestamps:    Timestamps are parsed into a list of datetime objects
# dataset.oxts:          List of OXTS packets and 6-dof poses as named tuples
# dataset.camN:          Returns a generator that loads individual images from camera N
# dataset.get_camN(idx): Returns the image from camera N at idx  
# dataset.gray:          Returns a generator that loads monochrome stereo pairs (cam0, cam1)
# dataset.get_gray(idx): Returns the monochrome stereo pair at idx  
# dataset.rgb:           Returns a generator that loads RGB stereo pairs (cam2, cam3)
# dataset.get_rgb(idx):  Returns the RGB stereo pair at idx  
# dataset.velo:          Returns a generator that loads velodyne scans as [x,y,z,reflectance]
# dataset.get_velo(idx): Returns the velodyne scan at idx  

# Get the following data:

#Length of the loaded dataset
len_dataset = None

#Gray stereo pair baseline
gray_baseline = None

#RGB stereo pair baseline
rgb_baseline = None

#Difference between the first and the second timestamp
diff_timestamp = None

#Last gray image (left camera)
last_gray_left = None

#Last gray image (right camera)
last_gray_right = None

#Last rgb image (left camera)
last_rgb_left = None

#Last rgb image (right camera)
last_rgb_right = None

#Third velodyne scan
third_velo = None

print('\nLength of the loaded dataset: ' + str(len_dataset))
print('\nGray stereo pair baseline [m]: ' + str(gray_baseline))
print('\nRGB stereo pair baseline [m]: ' + str(rgb_baseline))
print('\nDifference between Gray and RGB baselines: [m]', abs(rgb_baseline - gray_baseline))

print('\nDifference beteween the first and the second timestamp: ' + str(diff_timestamp))


f, ax = plt.subplots(2, 2, figsize=(15, 5))
ax[0, 0].imshow(last_gray_left, cmap='gray')
ax[0, 0].set_title('Last Gray Image (Left)')

ax[0, 1].imshow(last_gray_right, cmap='gray')
ax[0, 1].set_title('Last Gray Image (Right)')

ax[1, 0].imshow(last_rgb_left)
ax[1, 0].set_title('Last RGB Image (Left)')

ax[1, 1].imshow(last_rgb_right)
ax[1, 1].set_title('Right RGB Image (Right)')


f2 = plt.figure()
ax2 = f2.add_subplot(111, projection='3d')
# Plot every 100th point so things don't get too bogged down
velo_range = range(0, third_velo.shape[0], 100)
ax2.scatter(third_velo[velo_range, 0],
            third_velo[velo_range, 1],
            third_velo[velo_range, 2],
            c=third_velo[velo_range, 3],
            cmap='gray')
ax2.set_title('Third Velodyne scan (subsampled)')

In [ ]:
# Do stereo processing 
# Use last_gray_left and last_gray_right and calculate the disparity
disp_gray = None

# Use last_rgb_left and last_rgb_right and calculate the disparity
disp_rgb = None

# Display some data
f, ax = plt.subplots(2, 2, figsize=(15, 5))
ax[0, 0].imshow(last_gray_left, cmap='gray')
ax[0, 0].set_title('Left Gray Image (cam0)')

ax[0, 1].imshow(disp_gray, cmap='viridis')
ax[0, 1].set_title('Gray Stereo Disparity')

ax[1, 0].imshow(last_rgb_left)
ax[1, 0].set_title('Left RGB Image (cam2)')

ax[1, 1].imshow(disp_rgb, cmap='viridis')
ax[1, 1].set_title('RGB Stereo Disparity')

plt.show()